# Download XBRL

### 方針
- 提出書類情報（EDINET APIのresponce）をEDINET_responce.csvに保存（５年前まで遡りループで取得している）

（5年以上前は暗号化処理されており解凍できない）

- EDINETコードリストで業種を絞って、ＥＤＩＮＥＴコードを取得　（「提出者業種」と「上場区分」でフィルター）

EDINETコードリストは以下より

https://disclosure.edinet-fsa.go.jp/E01EW/BLMainController.jsp?uji.verb=W1E62071InitDisplay&uji.bean=ee.bean.W1E62071.EEW1E62071Bean&TID=W1E62071&PID=currentPage&SESSIONKEY=1562597359623&kbn=2&ken=0&res=0&idx=0&start=null&end=null&spf1=1&spf2=1&spf5=1&psr=1&pid=0&row=100&str=&flg=&lgKbn=2&pkbn=0&skbn=1&dskb=&askb=&dflg=0&iflg=0&preId=1

- EDINET_responce.csvをＥＤＩＮＥＴコード（edinetCode）とdocTypeCode(=120:有価証券報告書 or 140:四半期報告書)でフィルターし、ダウンロードするDocIDを取得

- EDINET APIのURLをDocIDで構成し、zipファイルをダウンロード

- （この後は、解凍、XBRLの取得、データベースへの保存）



### メモ

- start_dateは修正してください。また、../dataのディレクトリにEdinetcodeDlInfo.csv、EDINET_responce.csv、Docs(ダウンロードしたzipファイルの置き場)がある設定です。

- EDINET APIのアクセスは書類提出繁忙期（6月中〜下旬、8月上旬、11月上旬、2月上旬）の平日は避けた方が良さそう。

- EDINETのサーバーは深夜に応答が帰ってこないときがある。

- sleep timeはあまり短くするとEDINETのサーバーに負荷がかかり迷惑なので注意

### 参考
https://srbrnote.work/archives/1100

http://itref.fc2web.com/lang/python/edinet.html

ちなみに、XBRLパースは以下で公開されているスクリプトを利用しました。

https://srbrnote.work/archives/1430

## setup

In [1]:
import requests
import json
import pandas as pd
import numpy as np
import datetime
from time import sleep
import warnings
#warnings.filterwarnings('ignore')

### Request metadata

In [42]:
def EdinetQueryPD(params):
    EDINET_API_url = "https://disclosure.edinet-fsa.go.jp/api/v1/documents.json"
    res = requests.get(EDINET_API_url, params=params, verify=False)
    res_parsed=json.loads(res.text)
    responce_table=pd.read_json(json.dumps(res_parsed['results']))
    return responce_table



itr=0
start_date=datetime.datetime(2019,6,15) #5年前以降を入力してください
target_date=start_date + datetime.timedelta(days=itr)

params = {
    "date" : target_date.strftime("%Y-%m-%d"),
    "type" : 2
}
print(target_date)
output=EdinetQueryPD(params)
output.to_csv('../data/EDINET_responce.csv',encoding='utf-8') # 新規作成の場合
#output.to_csv('../data/EDINET_responce.csv',encoding='utf-8',mode='a',header=False) #追記の場合
sleep(1)
for itr in range(1,(datetime.datetime(2019,6,29)-start_date).days):
    
    target_date=start_date + datetime.timedelta(days=itr)
    print(target_date)
    params = {
        "date" : target_date.strftime("%Y-%m-%d"),
        "type" : 2
    }
    responce=EdinetQueryPD(params)
    responce.to_csv('../data/EDINET_responce.csv',encoding='utf-8',mode='a',header=False)
    sleep(1)
    

2019-06-15 00:00:00
2019-06-16 00:00:00
2019-06-17 00:00:00
2019-06-18 00:00:00
2019-06-19 00:00:00
2019-06-20 00:00:00
2019-06-21 00:00:00
2019-06-22 00:00:00
2019-06-23 00:00:00
2019-06-24 00:00:00
2019-06-25 00:00:00
2019-06-26 00:00:00
2019-06-27 00:00:00
2019-06-28 00:00:00


In [55]:
responce_data=pd.read_csv('../data/EDINET_responce.csv',encoding='utf-8')
responce_data.shape

(322349, 28)

In [44]:
responce_data.columns

Index(['Unnamed: 0', 'JCN', 'attachDocFlag', 'currentReportReason',
       'disclosureStatus', 'docDescription', 'docID', 'docInfoEditStatus',
       'docTypeCode', 'edinetCode', 'englishDocFlag', 'filerName', 'formCode',
       'fundCode', 'issuerEdinetCode', 'opeDateTime', 'ordinanceCode',
       'parentDocID', 'pdfFlag', 'periodEnd', 'periodStart', 'secCode',
       'seqNumber', 'subjectEdinetCode', 'submitDateTime',
       'subsidiaryEdinetCode', 'withdrawalStatus', 'xbrlFlag'],
      dtype='object')

In [57]:
responce_data.tail()

,Unnamed: 0,JCN,attachDocFlag,currentReportReason,disclosureStatus,docDescription,docID,docInfoEditStatus,docTypeCode,edinetCode,...,pdfFlag,periodEnd,periodStart,secCode,seqNumber,subjectEdinetCode,submitDateTime,subsidiaryEdinetCode,withdrawalStatus,xbrlFlag
322344,2267,5.180001e+12,0,第19条第2項第9号の2,0,臨時報告書,S100GDRB,0,180.0,E00079,...,1,NaN,NaN,18920.0,2268,NaN,2019-06-28 17:11,NaN,0,1
322345,2268,7.350001e+12,0,第19条第2項第9号の2,0,臨時報告書,S100GDQB,0,180.0,E00830,...,1,NaN,NaN,42160.0,2269,NaN,2019-06-28 17:13,NaN,0,1
322346,2269,5.240001e+12,0,NaN,0,訂正発行登録書,S100GDRF,0,90.0,E03585,...,1,NaN,NaN,83790.0,2270,NaN,2019-06-28 17:13,NaN,0,0
322347,2270,5.030001e+12,0,第19条第2項第9号の2,0,臨時報告書,S100GDKL,0,180.0,E03229,...,1,NaN,NaN,99970.0,2271,NaN,2019-06-28 17:14,NaN,0,1
322348,2271,6.140001e+12,0,NaN,0,確認書,S100GDLL,1,135.0,E04160,...,1,NaN,NaN,90830.0,2272,NaN,2019-06-28 15:47,NaN,0,0


### Filtering EDINET CODE

In [46]:
edinetcode=pd.read_csv('../data/EdinetcodeDlInfo.csv', \
                   header=1,index_col=False, engine="python", encoding="cp932")

In [47]:
mask=edinetcode.loc[:,'上場区分']=='上場'
edinetcode_filterd=edinetcode.loc[mask,:].copy()
classification=edinetcode_filterd.loc[:,'提出者業種'].values
print(np.unique(classification))

mask2=(edinetcode.loc[:,'提出者業種']=='食料品')&mask
edinetcode_filterd=edinetcode.loc[mask2,:].copy() #全データを落とす場合はこの行をコメントアウト
print(len(edinetcode_filterd))
edinetcodelist=edinetcode_filterd.loc[:,'ＥＤＩＮＥＴコード'].values

['その他製品' 'その他金融業' 'ガラス・土石製品' 'ゴム製品' 'サービス業' 'パルプ・紙' '不動産業' '保険業' '倉庫・運輸関連'
 '化学' '医薬品' '卸売業' '外国法人・組合' '小売業' '建設業' '情報・通信業' '機械' '水産・農林業' '海運業'
 '石油・石炭製品' '空運業' '精密機器' '繊維製品' '証券、商品先物取引業' '輸送用機器' '金属製品' '鉄鋼' '鉱業' '銀行業'
 '陸運業' '電気・ガス業' '電気機器' '非鉄金属' '食料品']
3746


### make spacified company edinet-codes.

In [58]:
mask=((responce_data.loc[:,'docTypeCode']==120)|(responce_data.loc[:,'docTypeCode']==140)).values
mask=(responce_data.loc[:,'edinetCode'].isin(edinetcodelist)).values * mask
mask.shape

(322349,)

### docID 

In [59]:
docIDs=responce_data.loc[mask,'docID'].values
len(docIDs) #全データはだいたい5万です。

50465

### Get document from EDINET

In [54]:
def GetDocEdinet(docid,url,params):

    filename = '../data/Docs/' + docid + '.zip'
    res = requests.get(url, params=params, verify=False)
    if res.status_code == 200:
      with open(filename, 'wb') as f:
        for chunk in res.iter_content(chunk_size=1024):
          f.write(chunk)

        
for itr in range(39587,len(docIDs)):
    print(itr)
    docid = docIDs[itr]
    url = 'https://disclosure.edinet-fsa.go.jp/api/v1/documents/' + docid
    params = {
      "type" : 1
    }
    GetDocEdinet(docid,url,params)
    sleep(6)

39587
39588
39589
39590
39591
39592
39593
39594
39595
39596
39597
39598
39599
39600
39601
39602
39603
39604
39605
39606
39607
39608
39609
39610
39611
39612
39613
39614
39615
39616
39617
39618
39619
39620
39621
39622
39623
39624
39625
39626
39627
39628
39629
39630
39631
39632
39633
39634
39635
39636
39637
39638
39639
39640
39641
39642
39643
39644
39645
39646
39647
39648
39649
39650
39651
39652
39653
39654
39655
39656
39657
39658
39659
39660
39661
39662
39663
39664
39665
39666
39667
39668
39669
39670
39671
39672
39673
39674
39675
39676
39677
39678
39679
39680
39681
39682
39683
39684
39685
39686
39687
39688
39689
39690
39691
39692
39693
39694
39695
39696
39697
39698
39699
39700
39701
39702
39703
39704
39705
39706
39707
39708
39709
39710
39711
39712
39713
39714
39715
39716
39717
39718
39719
39720
39721
39722
39723
39724
39725
39726
39727
39728
39729
39730
39731
39732
39733
39734
39735
39736
39737
39738
39739
39740
39741
39742
39743
39744
39745
39746
39747
39748
39749
39750
39751
39752
3975